In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('test.csv', encoding='ISO-8859-1')

In [ ]:
print(train_df.head())
print(test_df.head())

                                                text              target  \
0  python courses python courses, python exercise...  academic interests   
1  the learning point open digital education. a r...  academic interests   
2  tech news, latest technology, mobiles, laptops...  academic interests   
3  the best it certification materials in usa | k...  academic interests   
4  bioland scientific, for your research needs bi...  academic interests   

   Word Count  
0         125  
1         147  
2         143  
3         364  
4         176  
                                                text  Word Count      Index
0  equl offers enzyme assay kits, reagent mixture...         353  Article_0
1  gauthmath: instant math questions solver for f...         112  Article_1
2  Whats the No. 1 cause of blindness in older ad...         340  Article_2
3  Surfers will ride a wave in the Amazon this we...         465  Article_3
4  Why is the top of a leaf the most colorful, so...         269  A

In [ ]:

target_set = set(train_df['target'].unique())
print(target_set)
print(len(target_set))


{'academic interests', 'pets', 'shopping', 'careers', 'travel', 'health', 'style and fashion', 'real estate', 'music and audio', 'television', 'family and relationships', 'sports', 'hobbies and interests', 'healthy living', 'pharmaceuticals, conditions, and symptoms', 'automotives', 'news and politics', 'arts and culture', 'food and drinks', 'home and garden', 'personal finance', 'movies', 'video gaming', 'technology and computing', 'books and literature', 'business and finance'}
26


In [ ]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download the 'punkt' resource
nltk.download('punkt')

nltk.download('stopwords')

# Get the default English stop words
stop_words = set(stopwords.words('english'))

# Remove essential stop words
essential_words = {""}
stop_words = stop_words - essential_words

In [ ]:

import re
# 2. Text Cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove excess whitespace
    word_tokens = word_tokenize(text)  # Tokenize the text
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)
    # return text

# Apply the text cleaning function to the 'text' column
train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)

In [ ]:
# Save cleaned data to CSV (to offload memory)
train_data[['text', 'target']].to_csv('train_cleaned.csv', index=False)
test_data[['text','Index']].to_csv('test_cleaned.csv', index=False)

# Start


In [1]:
import pandas as pd

train_data = pd.read_csv("/content/drive/MyDrive/dataset/train_cleaned.csv")
test_data = pd.read_csv("/content/drive/MyDrive/dataset/test_cleaned.csv")

In [2]:

print("Train Data Info:")
train_data.info()
print("\nTest Data Info:")
test_data.info()

Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697527 entries, 0 to 697526
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    697527 non-null  object
 1   target  697527 non-null  object
dtypes: object(2)
memory usage: 10.6+ MB

Test Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174382 entries, 0 to 174381
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    174382 non-null  object
 1   Index   174382 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB


In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Vectorization using TF-IDF with more features and bigrams
tfidf = TfidfVectorizer(max_features=9000, ngram_range=(1,1))

X_train = tfidf.fit_transform(train_data['text'])
X_test = tfidf.transform(test_data['text'])

# 2. Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X_train, train_data['target'], test_size=0.2, random_state=42)

In [4]:
# 3. Model Training - MLPClassifier (Neural Network)
model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=20, verbose=True, batch_size=128, early_stopping=True)
model.fit(X_train, y_train)

# 4. Predictions on Validation Set
y_pred_val = model.predict(X_val)

# 5. Final Predictions on Test Set
y_test_pred = model.predict(X_test)

# 6. Prepare Submission File
submission = pd.DataFrame({'Index': test_data['Index'], 'target': y_test_pred})
submission.to_csv('submission.csv', index=False)


Iteration 1, loss = 1.00776269
Validation score: 0.768292
Iteration 2, loss = 0.68981434
Validation score: 0.783327
Iteration 3, loss = 0.56645134
Validation score: 0.789993
Iteration 4, loss = 0.46675756
Validation score: 0.792807
Iteration 5, loss = 0.37592521
Validation score: 0.790334
Iteration 6, loss = 0.29342417
Validation score: 0.785513
Iteration 7, loss = 0.21983977
Validation score: 0.781105
Iteration 8, loss = 0.15961435
Validation score: 0.777449
Iteration 9, loss = 0.11737469
Validation score: 0.775084
Iteration 10, loss = 0.09163250
Validation score: 0.771141
Iteration 11, loss = 0.07804189
Validation score: 0.771733
Iteration 12, loss = 0.07171462
Validation score: 0.771374
Iteration 13, loss = 0.06765309
Validation score: 0.769224
Iteration 14, loss = 0.06476510
Validation score: 0.768471
Iteration 15, loss = 0.06426690
Validation score: 0.770460
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [5]:
from sklearn import metrics
# y_pred_val = model.predict(X_val)
# Assuming you have 'actual' and 'predicted' variables defined with the true and predicted labels
score = max(0, 100*metrics.f1_score(y_val, y_pred_val, average='weighted'))

print(f"Score: {score}")

Score: 79.16920151163005
